In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import plotly.graph_objects as go
import datetime as dt

In [2]:
daily_ret=pd.read_csv('./daily_return.csv')
daily_ret.date=daily_ret.date.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d").date())
daily_ret=daily_ret.set_index('date')

price_all=pd.read_csv('./price_all.csv')
price_all.date=price_all.date.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d").date())
pivot_price=price_all[['date','code','close']].pivot(index='date', columns='code', values='close')
# pos_df=pd.DataFrame()

In [3]:
backtest_data=price_all.loc[price_all['date']<=dt.date(2020,5,1)]

fwd_data=price_all.loc[price_all['date']>=dt.date(2020,4,15)]

In [4]:
benchmark=(daily_ret.mean(axis=1)+ 1).cumprod().tolist()

In [5]:
# period=10
def profit_cal(stop_loss: float,take_profit: float, period: int,enter_mfi: float,exit_mfi: float, chart=False, backtest=True):
#     max_p=max([period_enter,period_exit])
    total_trade=0
    pos_df=pd.DataFrame()
    for stock in universe:
    #     print(stock)
        if backtest:
            price=backtest_data[backtest_data['code']==stock].reset_index(drop=True)
            pos_arr,trades=strategy_MFI(price_data=price, period=period, stop_loss=stop_loss,take_profit=take_profit,enter_mfi=enter_mfi, exit_mfi=exit_mfi)
            total_trade+=trades
            pos_df[stock]=pos_arr
        else:
            price=fwd_data[fwd_data['code']==stock].reset_index(drop=True)
            pos_arr,trades=strategy_MFI(price_data=price, period=period, stop_loss=stop_loss,take_profit=take_profit,enter_mfi=enter_mfi, exit_mfi=exit_mfi)
            total_trade+=trades
            pos_df[stock]=pos_arr
    if backtest:
        pos_df.index=backtest_data.date.unique().tolist()
        daily_ret_temp=daily_ret.loc[daily_ret.index<=dt.date(2020,5,1)]
    else:
        pos_df.index=fwd_data.date.unique().tolist()
        daily_ret_temp=daily_ret.loc[daily_ret.index>dt.date(2020,5,1)]
    
    max_holding=pos_df.count(axis=1).max()
    profit_arr=(daily_ret_temp*pos_df)
    daily_profit=profit_arr.mean(axis=1)
    cumulative=(daily_profit+1).cumprod().fillna(method='ffill')
    print(cumulative[-1], '_____________CUMULATIVE')
    print(max_holding, '_____________MAXIMUM HOLDING')
    print(total_trade, '_____________NO_OF TRADES')
    if chart:
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=profit_arr.index.tolist(), y=cumulative.tolist(),
                                 mode='lines',
                                 name='cumulative'))
        if backtest:
            fig.add_trace(go.Scatter(x=profit_arr.index.tolist(), y=benchmark[:1326],
                                     mode='lines',
                                     name='benchmark'))
        # if chart_title:
        #     fig.update_layout(title_text=chart_title)

        fig.show()
    return round(cumulative[-1],4), max_holding,total_trade,profit_arr

In [6]:
def strategy_MFI(price_data:pd.DataFrame, stop_loss: float, take_profit: float, period: int, enter_mfi: float,exit_mfi: float):
#     price_data=price_all[price_all['code']=='VNM'].reset_index(drop=True)
    price_data['typical_price']=(price_data['high']+price_data['low']+price_data['close'])/3
    price_data['change'] = price_data['typical_price']-price_data['typical_price'].shift(1)
    # price_data['up']=0
    price_data['positive_mf']=0
    price_data['negative_mf']=0
    price_data.loc[price_data['change']>0,'positive_mf']=price_data['typical_price']*price_data['volume']
    price_data.loc[price_data['change']<0,'negative_mf']=price_data['typical_price']*price_data['volume']

    price_data['MFI']=100-100/(1+price_data['positive_mf'].rolling(period).sum()/price_data['negative_mf'].rolling(period).sum())
    price_data['MFI'][period-1]=np.nan
    price_data['pos']=np.nan
#     price_data['MFI']

    
#     price_data['MFI']=(price_data['high']-price_data['low'])/price_data['volume']
    
    pos_arr, trades=[],0
    enter,enter_p, pos = 0, 0, np.nan
    for i in range(period, len(price_data)):
    #     print(i)
        price_data['pos'][i]=pos
        c, c1,c2,c_pre = price_data['close'][i],price_data['close'][i-1],price_data['close'][i-2],price_data['close'][i-5]
        v, v1, v2 = price_data['volume'][i], price_data['volume'][i-1],price_data['volume'][i-2]
        mfi=price_data['MFI'][i]
        o, h, l=price_data['open'][i], price_data['high'][i],price_data['low'][i]
        if np.isnan(pos) and mfi>enter_mfi and c>o and c>c_pre*1.05: #and c>c1 and v>v1>v2:        
#             print('ENTER___')
            pos=1
            enter=i
            enter_p=c
            trades+=1
        if pos==1 and i>=enter+3: 
            if c<c1 and c>enter_p*(1+take_profit):
                pos=np.nan                
            if mfi<exit_mfi and c<c1:
                pos=np.nan                
            if c<enter_p*(1-stop_loss):
                pos=np.nan
#                 print('CUT_LOSS____')
#     pos_arr=pd.Series(pos_arr).shift(period+1)
    price_data.to_csv('test_MFI.csv')
#     pos_arr=price_data['pos']
    return price_data['pos'],trades

In [7]:
a,b,c,d=profit_cal(stop_loss=0.05,take_profit=0.3, period=9,enter_mfi=90,exit_mfi=75,chart=True,backtest=True)

d.to_csv('test_profit.csv')

NameError: name 'universe' is not defined

In [ ]:
#VALIDATION____

price_data=backtest_data[backtest_data['code']=='DBC'].reset_index(drop=True)
a,b=strategy_MFI(price_data=price_data, stop_loss=0.05, take_profit=0.3, period=9, enter_mfi=90,exit_mfi=75)